In [5]:
# import random as rnd
# import secrets as scr
from datetime import datetime
from random import SystemRandom

rnd = SystemRandom()


class Lottery:

    def __init__(self,
                 max_numbers=90,
                 max_extra=90,
                 len_numbers=6,
                 len_extra=1):
        self.max_numbers = max_numbers
        self.max_extra = max_extra
        self.len_numbers = len_numbers
        self.len_extra = len_extra

    @property
    def combo(self): return self._combo

    @property
    def extra(self): return self._extra

    @property
    def backend(self): return self._backend.__name__

    @property
    def many(self): return self._many

    @staticmethod
    def choice(_len, max):
        numbers = list(range(1, max+1))

        def extraction():
            sample = rnd.choice(numbers)
            numbers.remove(sample)
            return sample

        combo = (extraction() for _ in range(_len))

        return frozenset(combo)

    @staticmethod
    def sample(_len, max):
        numbers = tuple(range(1, max+1))
        combo = rnd.sample(numbers, k=_len)

        return frozenset(combo)

    @staticmethod
    def randint(_len, max):
        combo = set()
        def draw(): return rnd.randint(1, max)

        # create an iterator that calls draw() until it returns 0, but
        # since it is impossible to get 0 from draw(), it works as an
        # infinite generator
        numbers = iter(draw, 0)

        while len(combo) < _len:
            combo.add(next(numbers))

        return frozenset(combo)

    def manySamples(self):
        '''
        To add further randomness, this method simulates several extractions, 
        among 1 and <size> times, and picks one casually ... hopefully 
        the winning one :D
        '''

        size = self._many or 1
        self._stop = rnd.randint(1, size)
    
        for _ in range(self._stop):
            sample = self.extract()

        return sample  # type: ignore

    def extract(self):
        combo = self._backend(self.len_numbers, self.max_numbers)
        extra = self._backend(self.len_extra, self.max_extra) or None

        return combo, extra

    def __call__(self, backend='choice', many=None):
        self._many = many
        self._backend = eval(backend,
                             {'__builtins__': {}},
                             {'choice': self.choice,
                              'randint': self.randint,
                              'sample': self.sample})

        self._combo, self._extra = self.manySamples()

        return self

    @property
    def draw(self):
        now = datetime.now().strftime("%d/%m/%Y %H:%M")

        print('Estrazione del:', now, '\nNumeri Estratti:',
              *sorted(self._combo))  # type: ignore

        if self._extra is not None:
            print('Superstar:', *sorted(self._extra))  # type: ignore

In [6]:
superenalotto = Lottery(max_numbers=90, max_extra=90, len_numbers=6, len_extra=0)
superenalotto(backend='choice', many=1_000_000).draw

Estrazione del: 02/07/2023 13:20 
Numeri Estratti: 11 13 16 30 44 74


In [4]:
superenalotto._stop

629686

In [54]:
eurojack = Lottery(max_numbers=50, max_extra=12, len_numbers=5, len_extra=2)
eurojack(backend='sample', many=500_000).draw

Estrazione del: 25/06/2023 10:22 
Numeri Estratti: 5 6 10 28 33
Superstar: 2 10


In [8]:
winForLife = Lottery(20, 20, 10, 1)
winForLife(backend='randint', many=500_000).draw

Estrazione del: 30/06/2023 12:00 
Numeri Estratti: 1 2 5 6 8 10 13 14 15 17
Superstar: 9


In [7]:
millionDay = Lottery(55, 0, 5, 0)
millionDay(many=100_000).draw

Estrazione del: 02/07/2023 13:22 
Numeri Estratti: 1 8 35 48 51


In [52]:
winForLife._stop

82424

In [55]:
superenalotto(backend='randint').draw

Estrazione del: 25/06/2023 10:22 
Numeri Estratti: 2 32 57 58 80 82


In [56]:
superenalotto.backend

'randint'

In [5]:
superenalotto.__dict__

{'max_numbers': 90,
 'max_extra': 90,
 'len_numbers': 6,
 'len_extra': 0,
 '_many': 1000000,
 '_backend': <function __main__.Lottery.choice(_len, max)>,
 '_stop': 629686,
 '_combo': frozenset({3, 31, 32, 67, 77, 87}),
 '_extra': None}

In [214]:
# for b in ['choice', 'int', 'sample']:
#    print('######')
#    eurojack(backend=b, many=0)
#    print('backend:', eurojack.backend)
#
# print('\n################# many ####################\n')
#
# for b in ['choice', 'int', 'sample']:
#    print('######')
#    eurojack(backend=b)
#    print('backend:', eurojack.backend)

In [2]:
win = Lottery(20, 20, 10, 1)

w_1 = frozenset({2, 3, 5, 7, 9, 11, 12, 14, 16, 18})
w_2 = frozenset({7})

In [3]:
i = 1
win()
x_1, x_2 = win.combo, win.extra
while ((w_1 != x_1) and (not w_1.isdisjoint(x_1))) or (w_2 != x_2):
    win()
    x_1, x_2 = win.combo, win.extra
    i += 1
    if i % 1_000_000 == 0:
        print(i)
print(i, '>>>', x_1, x_2)

490696 >>> frozenset({1, 4, 6, 8, 10, 13, 15, 17, 19, 20}) frozenset({7})
